In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import pickle
import re

In [2]:
#Hole alle Statistiken von einem Spiel
#Bis 2019 gehts, ab 2020 sieht die html seite anders aus
def statistik(url):
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    data=[]
    heim=[]
    gast=[]
    
    #Statistiken holen
    statistikheim = soup.find("div", {"class": "kick__data-grid--max-width kick__data-grid--max-width"}).findAll('div', attrs={'class':"kick__stats-bar__value kick__stats-bar__value--opponent1"})
    statistikgast = soup.find("div", {"class": "kick__data-grid--max-width kick__data-grid--max-width"}).findAll('div', attrs={'class':"kick__stats-bar__value kick__stats-bar__value--opponent2"})
    for i in statistikheim:
        splitter= str(i).split(">")
        while("" in splitter):
            splitter.remove("")
        zahlenheim=splitter[len(splitter)-1][:-5]
        heim.append(zahlenheim)
    #print(heim)
    for i in statistikgast:
        splitter= str(i).split(">")
        while("" in splitter):
            splitter.remove("")
        zahlengast= splitter[len(splitter)-1][:-5]
        gast.append(zahlengast)
        
    #Mannschaften holen
    mannschaften= url.split("/")
    
    mannschaft1= mannschaften[len(mannschaften)-2].split("-")
    team1=""
    for i in range(0,len(mannschaft1)-1):
        team1= team1+" "+mannschaft1[i]
    team1= team1.strip()

    mannschaft2= mannschaften[len(mannschaften)-1].split("-")
    team2=""
    for i in range(0,len(mannschaft2)-1):
        team2= team2+" "+mannschaft2[i]
    team2= team2.strip()
    
    data=[team1, team2, heim,gast] 
    
    return data

#Statistiken holen für die Jahre ab 2020
def statistik20(url):
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    data=[]
    heim=[]
    gast=[]
    mannschaften= soup.findAll("div", {"class":"kick__v100-gameCell__team__name"})
    heimmanschaft= mannschaften[0].getText()
    gastmannschaft= mannschaften[1].getText()
    
    #Statistiken holen
    statistikheim = soup.find("div", {"class": "kick__data-grid--max-width kick__data-grid--max-width"}).findAll('div', attrs={'class':"kick__stats-bar__value kick__stats-bar__value--opponent1"})
    statistikgast = soup.find("div", {"class": "kick__data-grid--max-width kick__data-grid--max-width"}).findAll('div', attrs={'class':"kick__stats-bar__value kick__stats-bar__value--opponent2"})
    for i in statistikheim:
        splitter= str(i).split(">")
        while("" in splitter):
            splitter.remove("")
        zahlenheim=splitter[len(splitter)-1][:-5]
        heim.append(zahlenheim)
    #print(heim)
    for i in statistikgast:
        splitter= str(i).split(">")
        while("" in splitter):
            splitter.remove("")
        zahlengast= splitter[len(splitter)-1][:-5]
        gast.append(zahlengast)
        
    
    data=[heimmanschaft, gastmannschaft, heim,gast] 
    
    return data


In [2]:
#Hole alle Spiele von einem Spieltag 

def spiele(link):
    req = requests.get(link)
    soup = BeautifulSoup(req.text, "html.parser")

    zahl= 0
    spieltag=[]
    for a in soup.find_all('a', href=True):
        if "/analyse" in a['href'] and zahl % 2==0:
            spieltag.append("https://www.kicker.de"+a['href'])
            zahl=zahl+1
        elif("/analyse" in a['href']):
            zahl=zahl+1
    return spieltag

#spiele(link= "https://www.kicker.de/1-bundesliga/spieltag/2017-18/1")


In [3]:
#Jedes Spiel von jedem Spieltag holen
zeile= ["Tore","Schuesse","Laufleistung(km)","Paesse","erfolgr. Paesse","Fehlpaesse","Passquote(%)","Ballbesitz(%)",
        "Zweikampfquote(%)","Foul","Gefoult","Abseits", "Ecken"]
reihe=["Spieltag", "Mannschaft1", "Mannschaft2", "Daten", "Statistik Heim", "Statistik Gast"]
def spieltage():
    
    file = open('Statistik2020.pkl', 'wb')
    # dump information to that file
    pickle.dump(reihe,file)
    
    zsm = pd.DataFrame(columns =reihe)
    counter=0
    for i in range(1,35):
        
        link= "https://www.kicker.de/1-bundesliga/spieltag/2019-20/"+str(i)
        games= spiele(link)
        firstgame=0
        for j in games:
            daten=[]
            statistic=[]
            j=j.replace("analyse","spieldaten")
            #print(i)
            daten.append(i)
            statistic= statistik20(j)
            counter2=0
            counter=counter+1
            print(j)
            for k in statistic:
                #print(k)
                if counter2==2:
                    daten.append(zeile)
                counter2= counter2+1
                daten.append(k)
            pickle.dump(daten,file)
            #print(daten)
    file.close()
    return counter
#spieltage()